## Using Direct Preference Optimization on a preference dataset


In [1]:
#%pip install --quiet transformers trl

In [1]:
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer

train_dataset = load_dataset('json', data_files="ad_preference_dataset.jsonl", split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [2]:
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B-Instruct")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct")

In [3]:
training_args = DPOConfig(output_dir="Qwen2-0.5B-DPO", logging_steps=10)
trainer = DPOTrainer(model=model, args=training_args, processing_class=tokenizer, train_dataset=train_dataset)
trainer.train()

Extracting prompt in train dataset:   0%|          | 0/460 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/460 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/460 [00:00<?, ? examples/s]

Step,Training Loss
10,0.681200
20,0.592400
30,0.593900
40,0.550600
50,0.462700
60,0.518600
70,0.222400
80,0.216900
90,0.239100
100,0.288800


TrainOutput(global_step=174, training_loss=0.31954808687341624, metrics={'train_runtime': 271.5562, 'train_samples_per_second': 5.082, 'train_steps_per_second': 0.641, 'total_flos': 0.0, 'train_loss': 0.31954808687341624, 'epoch': 3.0})

In [4]:
trainer.save_model(training_args.output_dir)

## Try out the trained model

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("Qwen2-0.5B-DPO")

In [6]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct")

In [7]:
from transformers import pipeline

generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [11]:
generator("You are writing a classified ad for used items in a neighborhood online forum. Write a one paragraph ad to sell an original sketch by Degas priced at $350", 
          max_length=100)

[{'generated_text': 'You are writing a classified ad for used items in a neighborhood online forum. Write a one paragraph ad to sell an original sketch by Degas priced at $350.\nAd: \n\n"Find an Original Sketch by Claude Monet for only $350! This exquisite painting, signed and numbered, was made during the last years of his life. It\'s a stunning representation of the French countryside and captures the essence of the natural world beautifully. The canvas measures approximately 8 feet x '}]